In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import factor_analyzer as factor

## Análisis Factorial con `factor_analizer`

Este es el código de la lectura de Dimensionalidad adaptado para que funcione con la librería `factor_analizer` versión 0.3.1

Con los cambios en la sintaxis esta librería es totalmente compatible con `scikit-learn`.

In [ ]:
df = pd.read_csv('cep45.csv')
df.head()

In [ ]:
# filtamos las columnas
trust_df = df.filter(regex="p17_*")
trust_df = trust_df.rename(columns={'p17_a': 'I.Catolica', 'p17_b': 'I.Evangelica', 'p17_c': 'FFAA', 'p17_d':
'Justicia',
'p17_e': 'Prensa', 'p17_f': 'Television', 'p17_g':'Sindicatos', 'p17_h':'Carabineros',
'p17_i': 'Gobierno', 'p17_j': 'PartidosPol', 'p17_k': 'Congreso', 'p17_l':'Empresas',
'p17_m':'Universidades', 'p17_n':'Radio'})

In [ ]:
import factor_analyzer.factor_analyzer as fact
fact.calculate_bartlett_sphericity(trust_df)

In [ ]:
fact.calculate_kmo(trust_df)

In [ ]:
factorize = fact.FactorAnalyzer(n_factors=2, method='principal')
factorize.fit(trust_df)
factor_screeplot = factorize.get_eigenvalues()[0]
plt.plot(np.arange(len(factor_screeplot)), factor_screeplot, 'o-', color='tomato')
plt.xlabel('Número de Factor')
plt.ylabel('Valores Propios')
plt.axhline(1)
plt.title("Scree plot");

In [ ]:
factor_loadings = pd.DataFrame(factorize.loadings_)
factor_loadings.set_index(trust_df.columns, inplace=True)
factor_loadings.columns = ['Factor1', 'Factor2']
factor_loadings

In [ ]:
factor_loadings.applymap(lambda x: np.where(x < .4, ' ', round(x, 2)))

In [ ]:
# guardemos la varianza explicada por cada dimensión
factor_var = factorize.get_factor_variance()
# cruzamos ambos factores
plt.plot(factor_loadings.Factor1, factor_loadings.Factor2,'o')
plt.xlim((-.2,1)) # definimos límites
plt.ylim((-.4, 1))
# Para cada elemento en los factores
for i, txt in enumerate(factor_loadings.index):
    # anotemos el nombre de la variable
    plt.annotate(txt, (factor_loadings['Factor1'][i],
    factor_loadings['Factor2'][i]))
# Grafiquemos la media de ambos factores
plt.axvline(factor_loadings['Factor1'].mean())
plt.axhline(factor_loadings['Factor2'].mean())
# Reportemos la varianza de cada dimensión
plt.xlabel('Factor 1: {}'.format(round(factor_var[0][1], 2)))
plt.ylabel('Factor 2: {}'.format(round(factor_var[1][1], 2)))

In [ ]:
factor_var = pd.DataFrame(np.asarray(factorize.get_factor_variance()))
factor_var.index = ['SS loadings (Variance)', 'Proportional variance', 'Cumulative variance']
factor_var.columns = ['Factor1', 'Factor2']
factor_var

In [ ]:
extract_factor_scores = factorize.transform(trust_df)
sns.distplot(extract_factor_scores[:, 0], label="Institucionalistas")
sns.distplot(extract_factor_scores[:, 1], label="Cotidianos")
plt.legend()